In [1]:
import numpy as np
# to compute angles
from math import atan2, cos, sin, sqrt, pi
import cv2
from PIL import Image

def convert_to_thresh(img):
  gray = cv2.cvtColor(np.array(img.convert('RGB'))[:, :, ::-1].copy(), cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]  
  thresh[thresh == 255] = 1
  return thresh

class MyImage:
  def __init__(self, src):
    self.raw = Image.open(src)
    self.thresh = convert_to_thresh(self.raw)
    self.contours, _ = cv2.findContours(self.thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

  def get_orientation(self, pts):
      sz = len(pts)
      data_pts = np.empty((sz, 2), dtype=np.float64)
      for i in range(data_pts.shape[0]):
          data_pts[i,0] = pts[i,0,0]
          data_pts[i,1] = pts[i,0,1]
      # Perform PCA analysis
      mean = np.empty((0))
      mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)

      cv2.circle(img, cntr, 3, (255, 0, 255), 2)
      p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
      p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
      draw_axis(img, cntr, p1, (0, 150, 0), 1)
      draw_axis(img, cntr, p2, (200, 150, 0), 5)
      angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
      return angle

  def plot_contours(self):
    for i,c in enumerate(self.contours):
      # area of each contour
      area = cv2.contourArea(c)

      # draw each contour only for visualization
      cv2.drawContours(self.thresh, self.contours, i, (0, 0, 255), 2)
      # find orientation of each shape
      print(self.get_orientation(c))

In [5]:
img = MyImage('neumes/elaphron/elaphron_2.png')
img.plot_contours()

0.06534072943271423
